In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# ============================================================
# USER PARAMETERS
# ============================================================

filename = "C:/Users/Labo/Desktop/199-trial2-control-cell-traces.csv"
output_dir = "C:/Users/Labo/Desktop/"

event_time = 337.387            # Entry time (s)
baseline_window = (-30, 0)      # Baseline window (s)
analysis_window = (-30, 90)     # Full window for plotting (s)
peak_window = (0, 30)           # Post-event response window (s)

min_duration_sec = 0.5          # Minimum response duration (s)

MIN_MAD = 1e-9                  # Floor to guard zero/near-zero MAD

# ============================================================
# LOAD DATA
# ============================================================

data = pd.read_csv(filename)

time = data.iloc[:, 0]
cell_columns = [c for c in data.columns if c.startswith("cell-")]
F = data[cell_columns]

# ============================================================
# TIME ALIGNMENT
# ============================================================

time_aligned = time - event_time

analysis_mask = (
    (time_aligned >= analysis_window[0]) &
    (time_aligned <= analysis_window[1])
)

if not analysis_mask.any():
    raise ValueError("No samples in analysis window; adjust analysis_window or event_time.")

time_win = time_aligned[analysis_mask]
F_win = F.loc[analysis_mask]

dt = np.median(np.diff(time_win))
if not np.isfinite(dt) or dt <= 0:
    raise ValueError("Invalid sampling interval detected; check the time vector.")

min_frames = max(1, int(np.ceil(min_duration_sec / dt)))

# ============================================================
# MASKS
# ============================================================

baseline_mask = (
    (time_win >= baseline_window[0]) &
    (time_win < baseline_window[1])
)

peak_mask = (
    (time_win >= peak_window[0]) &
    (time_win <= peak_window[1])
)

if not baseline_mask.any():
    raise ValueError("Baseline window has no samples; adjust baseline_window.")

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def mad_normalize(signal, baseline_mask, min_mad=MIN_MAD):
    baseline = signal[baseline_mask]
    baseline = baseline[np.isfinite(baseline)]
    if baseline.size == 0:
        raise ValueError("Baseline window is empty or contains only non-finite values.")

    med = np.median(baseline)
    mad = np.median(np.abs(baseline - med))

    if mad <= min_mad:
        fallback_std = np.std(baseline, ddof=1)
        if np.isfinite(fallback_std) and fallback_std > min_mad:
            mad = fallback_std
        else:
            mad = min_mad

    return (signal - med) / mad, med, mad


def sustained_response(mask, min_len):
    count = 0
    for v in mask:
        count = count + 1 if v else 0
        if count >= min_len:
            return True
    return False


def classify_and_stats(Z, baseline_mask, peak_mask):
    baseline_Z = Z[baseline_mask]
    peak_Z = Z[peak_mask]

    baseline_Z = baseline_Z[np.isfinite(baseline_Z)]
    peak_Z = peak_Z[np.isfinite(peak_Z)]

    if baseline_Z.size == 0 or peak_Z.size == 0:
        return "Insufficient data", {
            "Baseline median (z)": np.nan,
            "Baseline std (z)": np.nan,
            "Upper threshold (med+2*std)": np.nan,
            "Lower threshold (med-2*std)": np.nan,
            "Max peak (0-30s)": np.nan,
            "Min peak (0-30s)": np.nan,
            "Baseline median (raw)": np.nan,
            "Baseline MAD (raw)": np.nan,
        }

    base_med = np.median(baseline_Z)
    base_std = np.std(baseline_Z, ddof=1)
    if not np.isfinite(base_std) or base_std <= 0:
        base_std = MIN_MAD  # keep thresholds finite

    upper_thr = base_med + 2 * base_std
    lower_thr = base_med - 2 * base_std

    excited_mask = peak_Z > upper_thr
    inhibited_mask = peak_Z < lower_thr

    excited = sustained_response(excited_mask, min_frames)
    inhibited = sustained_response(inhibited_mask, min_frames)

    max_peak = np.max(peak_Z) if peak_Z.size else np.nan
    min_peak = np.min(peak_Z) if peak_Z.size else np.nan

    if excited and inhibited:
        response = "Ambiguous (excited and inhibited)"
    elif excited:
        response = "Cell Excited"
    elif inhibited:
        response = "Cell Inhibited"
    else:
        response = "Cell Not Responsive"

    stats = {
        "Baseline median (z)": base_med,
        "Baseline std (z)": base_std,
        "Upper threshold (med+2*std)": upper_thr,
        "Lower threshold (med-2*std)": lower_thr,
        "Max peak (0-30s)": max_peak,
        "Min peak (0-30s)": min_peak,
    }

    return response, stats


def plot_cell(time, Z, cell_name, response, stats):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=time,
        y=Z,
        mode="lines",
        line=dict(width=1),
        name="MAD-normalized signal"
    ))

    fig.add_shape(
        type="line",
        x0=0, x1=0,
        y0=np.nanmin(Z), y1=np.nanmax(Z),
        line=dict(color="black", dash="dot", width=2)
    )

    annotation_text = (
        f"{response} | "
        f"Baseline(z): {stats['Baseline median (z)']:.2f} (std {stats['Baseline std (z)']:.2f}) | "
        f"Thr+: {stats['Upper threshold (med+2*std)']:.2f} | "
        f"Thr-: {stats['Lower threshold (med-2*std)']:.2f} | "
        f"Max(0-30s): {stats['Max peak (0-30s)']:.2f} | "
        f"Min(0-30s): {stats['Min peak (0-30s)']:.2f}"
    )

    x_mid = 0.5 * (analysis_window[0] + analysis_window[1])
    y_top = np.nanmax(Z)

    fig.add_annotation(
        x=x_mid,
        y=y_top,
        text=annotation_text,
        showarrow=False,
        align="center",
        font=dict(size=12),
        bgcolor="rgba(255,255,255,0.7)",
        xanchor="center",
        yanchor="top"
    )

    fig.update_layout(
        title=f"{cell_name} | Event-aligned response",
        xaxis_title="Time from entry (s)",
        yaxis_title="MAD-normalized signal",
        template="simple_white"
    )

    fig.update_xaxes(range=list(analysis_window))

    fig.show()

# ============================================================
# MAIN LOOP
# ============================================================

Z_dict = {}
response_dict = {}

for cell in cell_columns:
    signal = F_win[cell].values

    try:
        Z, baseline_med_raw, baseline_mad_raw = mad_normalize(signal, baseline_mask)
    except ValueError as err:
        response = f"Skipped: {err}"
        stats = {
            "Baseline median (z)": np.nan,
            "Baseline std (z)": np.nan,
            "Upper threshold (med+2*std)": np.nan,
            "Lower threshold (med-2*std)": np.nan,
            "Max peak (0-30s)": np.nan,
            "Min peak (0-30s)": np.nan,
            "Baseline median (raw)": np.nan,
            "Baseline MAD (raw)": np.nan,
        }
        Z = np.full_like(time_win, np.nan, dtype=float)
    else:
        response, stats = classify_and_stats(Z, baseline_mask, peak_mask)
        stats["Baseline median (raw)"] = baseline_med_raw
        stats["Baseline MAD (raw)"] = baseline_mad_raw

    Z_dict[cell] = Z
    response_dict[cell] = response

    plot_cell(time_win, Z, cell, response, stats)

# ============================================================
# SAVE OUTPUTS
# ============================================================

zscore_df = pd.DataFrame(Z_dict)
zscore_df.insert(0, "time", time_win.values)

zscore_file = os.path.join(
    output_dir,
    "199-trial2-control-nonparametric-zscore.csv"
)
zscore_df.to_csv(zscore_file, index=False)

response_file = os.path.join(
    output_dir,
    "199-trial2-control-nonparametric-response_classification.csv"
)

pd.DataFrame.from_dict(
    response_dict, orient="index", columns=["Response"]
).to_csv(response_file)

print("Analysis complete.")
print(f"Z-score data saved to: {zscore_file}")
print(f"Response classification saved to: {response_file}")


Analysis complete.
Z-score data saved to: C:/Users/Labo/Desktop/199-trial2-control-nonparametric-zscore.csv
Response classification saved to: C:/Users/Labo/Desktop/199-trial2-control-nonparametric-response_classification.csv
